# Time Series in Anomaly Detection

## Exercise

### Imports

In [ ]:
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd

# from matplotlib import style
import seaborn as sns
%matplotlib inline

# pd.plotting.register_matplotlib_converters()

### Acquire

In [ ]:
colnames = ['date', 'endpoint', 'user_id', 'cohort_id', 'source_ip']
df = pd.read_csv("anonymized-curriculum-access.txt", 
                 sep="\s", 
                 header=None, 
                 names = colnames, 
                 usecols=[0, 2, 3, 4, 5])
df.head()


### Prepare

In [ ]:
df.date = pd.to_datetime(df.date)
df = df.set_index(df.date)
pages = df['endpoint'].resample('d').count()
pages.head()

#### EMA

In [ ]:
# 7 day EMA
ema_7d = pages.ewm(span=7).mean()
ema_7d.head()

In [ ]:
# 14 days EMA
ema_14d = pages.ewm(span=14).mean()
ema_14d.head()

In [ ]:
# 30 days EMA
ema_30d = pages.ewm(span=30).mean()
ema_30d.head()

In [ ]:
# 90 days EMA
ema_90d = pages.ewm(span=90).mean()
ema_90d.head()

In [ ]:
fig, ax = plt.subplots(figsize=(12,6))

ax.plot(pages.index, pages, label='Daily', alpha=.5)

ax.plot(pages.index, ema_7d, label = '7-day EMA')
ax.plot(pages.index, ema_14d, label = '14-day EMA')
ax.plot(pages.index, ema_30d, label = '30-day EMA')
ax.plot(pages.index, ema_90d, label = '30-day EMA')

ax.legend(loc='best')
ax.set_ylabel('Number of pages')

plt.show()

Bollinger Bands

- a volatility indicator and commonly used in stock market trading.

- Made up of 3 lines, the Upper Band (UB), the Lower Band (LB) and the Midband.

Midband

- The Exponential Moving Average

midband = train.ewm(span=30).mean()

Upper & Lower Band

- UB/LB = Midband +/- stdev * K

- stdev = train.ewm(span=30).std()

- K = the number of standard deviations to go up and down from the EMA

%b, Percent Bandwidth

- Shows where the last value sits in relation to the bands

- %b = (last−LB) / (UB−LB)

- %b > 1 => point lies above UB

- %b < 0 => point lies below LB

- %b == .5 => point lies on the midband.

Bandwidth

- The width of the bands 

- (UB - LB) / Midband

In [ ]:
# set the window span
span = 30

# compute midband
midband = pages.ewm(span=span).mean()

midband.head()

In [ ]:
# compute exponential stdev
stdev = pages.ewm(span=span).std()

stdev.head()

In [ ]:
# compute upper and lower bands
ub = midband + stdev*3
lb = midband - stdev*3
ub

In [ ]:
# concatenate ub and lb together into one df, bb
bb = pd.concat([ub, lb], axis=1)

bb.columns = ['ub', 'lb']
bb.head()

In [ ]:
my_df = pd.concat([pages, midband, bb], axis=1)
my_df.columns = ['pages', 'midband', 'ub', 'lb']
my_df.head()

In [ ]:
fig, ax = plt.subplots(figsize=(12,8))

ax.plot(my_df.index, my_df.pages, label='Number of Pages')

ax.plot(my_df.index, my_df.midband, label = '30-day EMA/midband')
ax.plot(my_df.index, my_df.ub, label = 'Upper Band')
ax.plot(my_df.index, my_df.lb, label = 'Lower Band')

ax.legend(loc='best')
ax.set_ylabel('Number of pages')

plt.show()

In [ ]:
# get the last measure of gb
last_measure = my_df.iloc[-1].pages
last_measure

In [ ]:
# get the last measure of gb
last_measure = my_df.iloc[-1].pages
last_measure

In [ ]:
# get the upper band value on the last date
last_ub = my_df.iloc[-1].ub
last_ub

In [ ]:
# compute %b
last_pct_b = (last_measure - last_lb)/(last_ub - last_lb)

print('%b for last datapoint: ', round(last_pct_b, 2))

In [ ]:
my_df.tail(3)

In [ ]:
my_df['pct_b'] = (my_df['pages'] - my_df['lb'])/(my_df['ub'] - my_df['lb'])

my_df.head()

In [ ]:
my_df[my_df['pct_b']>1]

In [ ]:
def acquire(file_name, column_names):
    return pd.read_csv(file_name, sep="\s", header=None, names=column_names, usecols=[0, 2, 3, 4, 5])

def prep(df, user):
    df = df[df.user_id == user]
    df.date = pd.to_datetime(df.date)
    df = df.set_index(df.date)
    pages = df['endpoint'].resample('d').count()
    return pages

def compute_pct_b(pages, span, weight, user):
    midband = pages.ewm(span=span).mean()
    stdev = pages.ewm(span=span).std()
    ub = midband + stdev*weight
    lb = midband - stdev*weight
    bb = pd.concat([ub, lb], axis=1)
    my_df = pd.concat([pages, midband, bb], axis=1)
    my_df.columns = ['pages', 'midband', 'ub', 'lb']
    my_df['pct_b'] = (my_df['pages'] - my_df['lb'])/(my_df['ub'] - my_df['lb'])
    my_df['user_id'] = user
    return my_df

def plt_bands(my_df, user):
    fig, ax = plt.subplots(figsize=(12,8))
    ax.plot(my_df.index, my_df.pages, label='Number of Pages, User: '+str(user))
    ax.plot(my_df.index, my_df.midband, label = 'EMA/midband')
    ax.plot(my_df.index, my_df.ub, label = 'Upper Band')
    ax.plot(my_df.index, my_df.lb, label = 'Lower Band')
    ax.legend(loc='best')
    ax.set_ylabel('Number of Pages')
    plt.show()

def find_anomalies(df, user, span, weight):
    pages = prep(df, user)
    my_df = compute_pct_b(pages, span, weight, user)
    # plt_bands(my_df, user)
    return my_df[my_df.pct_b>1]


In [ ]:
file_name = "anonymized-curriculum-access.txt"
column_names = ['date', 'endpoint', 'user_id', 'cohort_id', 'source_ip']
df = acquire(file_name, column_names)
df.head()


In [ ]:
span = 30
weight = 3.5

anomalies = pd.DataFrame()
for u in list(df.user_id.unique()):
    user_df = find_anomalies(df, u, span, weight)
    anomalies = pd.concat([anomalies, user_df], axis=0)


## Notes

### Things I missed

- everything before bollanger band